In [1]:
import pandas as pd
from glob import glob
import os
import time
from tqdm import tqdm

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
main_path = 'C:/Users/Admin/바탕 화면/다운로드/정형철교수님/'

발생 = '지역별_질병별_월별_감염병전체전체_지역전체전체(2001.01-2022.06)/'
사망 = '지역별_사망_월별_감염병전체전체_지역(2001-2021)/'
성별 = '지역별_질병별_연도별_성별_연령별_발생/'

# labeling을 위한 작업

In [21]:
## 시도lst

시도lst = ['서울', '부산', '대구', '인천',
      '광주', '대전', '울산', '경기', '강원', 
      '충북', '충남', '전북', '전남', '경북',
      '경남', '제주', '세종']
len(시도lst) # 17

## name2병id (len 63)

급1병_lst = ['에볼라바이러스병', '마버그열', '라싸열', '크리미안콩고출혈열', '남아메리카출혈열', '리프트밸리열', '두창', '페스트',
            '탄저', '보툴리눔독소증', '야토병', '중증급성호흡기증후군(SARS)', '중동호흡기증후군(MERS)', '동물인플루엔자 인체감염증',
            '신종인플루엔자', '디프테리아']

급2병_lst = ['수두', '홍역', '콜레라', '장티푸스', '파라티푸스', '세균성이질', '장출혈성대장균감염증', 'A형간염', '백일해',
            '유행성이하선염', '풍진(2018년이전)', '풍진(선천성)', '풍진(후천성)', '폴리오', '수막구균 감염증',
            'b형헤모필루스인플루엔자', '폐렴구균 감염증', '한센병', '성홍열', '반코마이신내성황색포도알균(VRSA) 감염증',
            '카바페넴내성장내세균속균종(CRE) 감염증', 'E형간염']

급3병_lst = ['파상풍', 'B형간염', '일본뇌염', 'C형간염', '말라리아', '레지오넬라증', '비브리오패혈증', '발진티푸스', '발진열',
             '쯔쯔가무시증', '렙토스피라증', '브루셀라증', '공수병', '신증후군출혈열',
             '크로이츠펠트-야콥병(CDJ) 및 변종크로이츠펠트-야콥병(vCJD)', '황열', '뎅기열', '큐열', '웨스트나일열', '라임병',
             '진드기매개뇌염', '유비저', '치쿤구니야열', '중증열성혈소판감소증후군(SFTS)', '지카바이러스감염증']

병name_lst = 급1병_lst + 급2병_lst + 급3병_lst
len(병name_lst) # 63

병id_lst = [i for i in range(len(급1병_lst))] + [i for i in range(len(급2병_lst))] + [i for i in range(len(급3병_lst))]

name2병id = {병name_lst[i] : 병id_lst[i] for i in range(len(병id_lst))}
len(name2병id) # 63

급id_match_lst = [1]*len(급1병_lst) + [2]*len(급2병_lst) + [3]*len(급3병_lst)


## 지역 (len 272)
군구서울 = ['강남구', '강동구', '강북구', '강서구', '관악구', '광진구', '구로구', '금천구', '노원구', '도봉구', '동대문구', '동작구', '마포구', '서대문구', '서초구', '성동구', '성북구', '송파구', '양천구', '영등포구', '용산구', '은평구',
           '종로구', '중구', '중랑구']

군구부산 = ['강서구', '금정구', '기장군', '남구', '동구', '동래구', '부산진구', '북구', '사상구', '사하구', '서구', '수영구', '연제구', '영도구', '중구', '해운대구']

군구대구 = ['남구', '달서구', '달성군', '동구', '북구', '서구', '수성구', '중구']

군구인천 = ['강화군', '계양구', '미추홀구', '남동구', '동구', '부평구', '서구', '연수구', '옹진군', '중구']

군구광주 = ['광산구', '남구', '동구', '북구', '서구']

군구대전 = ['대덕구', '동구', '서구', '유성구', '중구']

군구울산 = ['남구', '동구', '북구', '울주군', '중구']

군구경기 = ['가평군', '고양시 덕양구', '고양시 일산동구', '고양시 일산서구', '과천시', '광명시', '광주시', '구리시', '군포시', '김포시', '남양주시', '동두천시', '부천시 소사구', '부천시 오정구', '부천시 원미구', '성남시 분당구', '성남시 수정구',
           '성남시 중원구', '수원시 권선구', '수원시 영통구', '수원시 장안구', '수원시 팔달구', '시흥시', '안산시 단원구', '안산시 상록구', '안성시', '안양시 동안구', '안양시 만안구', '양주시', '양평군', '여주시', '연천군', '오산시', '용인시 기흥구', '용인시 수지구',
           '용인시 처인구', '의왕시', '의정부시', '이천시', '파주시', '평택시', '포천시', '하남시', '화성시', '양주군', '고양시 일산구', '포천군', '용인시', '남양주', '고양시 일산동', '안산시', '부천시']

군구강원 = ['강릉시', '고성군', '동해시', '삼척시', '속초시', '양구군', '양양군', '영월군', '원주시', '인제군', '정선군', '철원군', '춘천시', '태백시', '평창군', '홍천군', '화천군', '횡성군']

군구충북 = ['괴산군', '단양군', '보은군', '영동군', '옥천군', '음성군', '제천시',' 증평군', '진천군', '청원군', '(구)청주시 상당구', '(구)청주시 흥덕구', '충주시', '청주시 상당구', '청주시 흥덕구', '청주시 서원구', '청주시 청원구']

군구충남 = ['계룡시', '공주시', '금산군', '논산시', '당진시', '보령시', '부여군', '서산시', '서천군', '아산시', '연기군', '예산군', '천안시', '청양군', '태안군', '홍성군', '천안시 동남구', '천안시 서북구']

군구전북 = ['고창군', '군산시' ,'김제시', '남원시', '무주군', '부안군', '순창군', '완주군', '익산시', '임실군', '장수군', '전주시 덕진구', '전주시 완산구', '정읍시', '진안군', '전주시']

군구전남 = ['강진군', '고흥군', '곡성군', '광양시', '구례군', '나주시', '담양군', '목포시', '무안군', '보성군', '순천시', '신안군', '여수시', '영광군', '영암군', '완도군', '장성군', '장흥군', '진도군', '함평군', '해남군', '화순군']

군구경북 = ['경산시', '경주시', '고령군', '구미시', '군위군', '김천시', '문경시', '봉화군', '상주시', '성주군', '안동시', '영덕군', '영양군', '영주시', '영천시', '예천군', '울릉군', '울진군', '의성군', '청도군', '청송군', '칠곡군',
           '포항시 남구', '포항시 북구', '포항시']

군구경남 = ['거제시', '거창군', '고성군', '김해시', '남해군', '마산시', '밀양시', '사천시', '산청군', '양산시', '의령군', '진주시', '진해시', '창녕군', '창원시', '통영시', '하동군', '함안군', '함양군', '합천군', '창원시 마산합포구', '창원시 마산회원구',
           '창원시 성산구', '창원시 의창구', '창원시 진해구']

군구제주 = ['서귀포시', '제주시', '남제주군', '북제주군']

군구세종 = ['세종시']



### 각 시도별 군구 길이
# for i in 시도lst:
#     i, len(globals()[f'군구{i}'])


### 모든 시도의 모든 군구의 수
군구all = 군구서울 + 군구부산 + 군구대구 + 군구인천 + 군구광주 + 군구대전 + 군구울산 + 군구경기 + 군구강원 + 군구충북 + 군구충남 + 군구전북 + 군구전남 + 군구경북 + 군구경남 + 군구제주 + 군구세종
len(군구all) # 272

17

63

63

272

In [8]:
17*21 # 사망 폴더 내 파일 개수 = 17개 시도 * 21년 = 357

357

# 발생, 사망, 성별

## 발생

In [74]:
## 공통
file_list = glob(main_path+발생+'*')
len(file_list) # 258
file_list = sorted(file_list) # default가 오름차순 정렬이긴 하지만...


########################################################################## 바뀜
## 발생 columns
발생cols = ['시도', '군구'] + 병name_lst + ['연도', '월']

## df 틀
fin_dat = pd.DataFrame(columns = 발생cols)
fin_dat

258

,시도,군구,에볼라바이러스병,마버그열,라싸열,크리미안콩고출혈열,남아메리카출혈열,리프트밸리열,두창,페스트,...,큐열,웨스트나일열,라임병,진드기매개뇌염,유비저,치쿤구니야열,중증열성혈소판감소증후군(SFTS),지카바이러스감염증,연도,월


In [75]:
for i in tqdm(file_list):
    
    ## Data Load
    tmp_dat = pd.read_csv(i, encoding = 'cp949', header = None)
    
    ########################################################################## 바뀜
    ## drop 계, NaN - 시도==군구인 식으로 합계 정보 나타나있었음.
    tmp_dat.drop(tmp_dat[tmp_dat.iloc[:, 0]==tmp_dat.iloc[:, 1]].index, inplace =  True)
    tmp_dat = tmp_dat.iloc[:, :-1]
    
    ########################################################################## 바뀜
    ## 식별열
    YY = i.split('_')[-1].split('-')[0]
    MM = i.split('_')[-1].split('-')[-1].split('.')[0]
#     YY, MM
    tmp_dat['연도'] = YY
    tmp_dat['월'] = MM
    
    ########################################################################## 바뀜
    ## col name chng
    tmp_dat.columns = 발생cols
    
    ## concat
    fin_dat = pd.concat([fin_dat, tmp_dat], axis = 0, ignore_index = True)
    
#     fin_dat
    
#     break

100%|████████████████████████████████████████████████████████████████████████████████| 258/258 [00:03<00:00, 78.54it/s]


In [147]:
# fin_dat # shape = (70176, 67)

In [89]:
fin_dat.shape[0] / len(file_list) # file개수로 나누면 군구 개수 == 한 파일 당 군구 수

272.0

### 저장

In [78]:
# fin_dat.to_csv('발생2001_01-2022_06.csv', index = False)

In [150]:
확인 = pd.read_csv('발생2001_01-2022_06.csv')
확인.shape # (70176, 67)
확인.head()

(70176, 67)

,시도,군구,에볼라바이러스병,마버그열,라싸열,크리미안콩고출혈열,남아메리카출혈열,리프트밸리열,두창,페스트,...,큐열,웨스트나일열,라임병,진드기매개뇌염,유비저,치쿤구니야열,중증열성혈소판감소증후군(SFTS),지카바이러스감염증,연도,월
0,서울,강남구,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2001,1
1,서울,강동구,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2001,1
2,서울,강북구,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2001,1
3,서울,강서구,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2001,1
4,서울,관악구,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2001,1


## 사망

In [94]:
## 공통
file_list = glob(main_path+사망+'*')
len(file_list) # 357
file_list = sorted(file_list) # default가 오름차순 정렬이긴 하지만...


########################################################################## 바뀜
## 사망 columns
월 = [f'{i}월' for i in range(1, 13)]
사망cols = ['급수', '병'] + 월 + ['연도', '군구']
len(사망cols)

## df 틀
fin_dat = pd.DataFrame(columns = 사망cols)
fin_dat

357

16

,급수,병,1월,2월,3월,4월,5월,6월,7월,8월,9월,10월,11월,12월,연도,군구


In [95]:
for i in tqdm(file_list):
    
    ## Data Load
    tmp_dat = pd.read_csv(i, encoding = 'cp949', header = None)
    
    ######################################################################### 바뀜
    ## drop 계, NaN
    tmp_dat.drop(tmp_dat.columns[2], axis = 1, inplace = True)
    tmp_dat = tmp_dat.iloc[:, :-1]
    
    ######################################################################### 바뀜
    ## 식별열
    YY = i.split('_')[-1].split('-')[0]
    군구nm = i.split('_')[-1].split('-')[-1].split('.')[0]
#     YY, 군구nm
    tmp_dat['연도'] = YY
    tmp_dat['군구'] = 군구nm
    
    ######################################################################### 바뀜
    ## col name chng
    tmp_dat.columns = 사망cols
    
    ## concat
    fin_dat = pd.concat([fin_dat, tmp_dat], axis = 0, ignore_index = True)
    
#     if tmp_dat.shape[0] != 63:
#         print(i)
    
#     break

100%|███████████████████████████████████████████████████████████████████████████████| 357/357 [00:01<00:00, 320.95it/s]


In [148]:
# fin_dat # shape = (22491, 16)

In [98]:
fin_dat.shape[0] / len(file_list) # 파일 개수(357)로 나눴는데 나누어 안떨어짐(질병개수 63이 안뜸.)...;;; -> 알고보니 fin_dat을 리셋 안해주고 진행해서 그랬던 것이었음.

63.0

### 저장

In [101]:
# fin_dat.to_csv('사망2001-2021.csv', index = False)

In [103]:
확인 = pd.read_csv('사망2001-2021.csv')
확인.shape # (22491, 16)
확인.head()

(22491, 16)

,급수,병,1월,2월,3월,4월,5월,6월,7월,8월,9월,10월,11월,12월,연도,군구
0,1급,에볼라바이러스병,-,-,-,-,-,-,-,-,-,-,-,-,2001,강원
1,1급,마버그열,-,-,-,-,-,-,-,-,-,-,-,-,2001,강원
2,1급,라싸열,-,-,-,-,-,-,-,-,-,-,-,-,2001,강원
3,1급,크리미안콩고출혈열,-,-,-,-,-,-,-,-,-,-,-,-,2001,강원
4,1급,남아메리카출혈열,-,-,-,-,-,-,-,-,-,-,-,-,2001,강원


## 성별
- 병 급수, 군구 명 다 나중에 mapping 해주기로

In [139]:
dir_list = glob(main_path+성별+'*')
len(dir_list) # 68 # 정렬 필요 없음.


## 성별 columns
연도 = [f'{i}년' for i in range(2001, 2022)]
성별cols = ['연령단위', '성별'] + 연도 + ['병', '군구']
len(성별cols)

## df 틀
fin_dat = pd.DataFrame(columns = 성별cols)
fin_dat


for dir_ in tqdm(dir_list):
    file_list = glob(dir_+'/'+'*')
#     len(file_list) # 272
    file_list = sorted(file_list) # default가 오름차순 정렬이긴 하지만...
    
    for i in file_list:
    
        ## Data Load
        tmp_dat = pd.read_csv(i, encoding = 'cp949', header = None)
        
        ########################################################################## 바뀜
        ## drop 계, NaN
        tmp_dat.drop(tmp_dat[(tmp_dat.iloc[:, 0]== '계') | (tmp_dat.iloc[:, 1]== '계')].index, inplace =  True)
        tmp_dat = tmp_dat.iloc[:, :-1]
        
        
        ########################################################################## 바뀜
        ## 식별열
        병nm = i.split('_')[-1].split(' ')[0]
        군구nm = i.split('_')[-1].split(' ')[-1].split('.')[0][1:-1]
#         병nm, 군구nm
        tmp_dat['병'] = 병nm
        tmp_dat['군구'] = 군구nm
        
        ########################################################################## 바뀜
        # col name chng
        tmp_dat.columns = 성별cols

        ## concat
        fin_dat = pd.concat([fin_dat, tmp_dat], axis = 0, ignore_index = True)
        
#         fin_dat
#         break
#     break


########################################################################## 바뀜


63

25

,연령단위,성별,2001년,2002년,2003년,2004년,2005년,2006년,2007년,2008년,...,2014년,2015년,2016년,2017년,2018년,2019년,2020년,2021년,병,군구


100%|██████████████████████████████████████████████████████████████████████████████████| 63/63 [07:31<00:00,  7.16s/it]


In [149]:
# fin_dat # shape = (479808, 25)

In [142]:
fin_dat.shape[0] / (len(dir_list) * len(file_list))
479808 / (272*63) # 이러면 행 수 나옴

28.0

28.0

In [143]:
fin_dat.head()

,연령단위,성별,2001년,2002년,2003년,2004년,2005년,2006년,2007년,2008년,...,2014년,2015년,2016년,2017년,2018년,2019년,2020년,2021년,병,군구
0,0~4세,남,-,-,-,-,-,-,-,-,...,0,0,0,0,0,0,0,0,에볼라바이러스병,001
1,0~4세,여,-,-,-,-,-,-,-,-,...,0,0,0,0,0,0,0,0,에볼라바이러스병,001
2,5~9세,남,-,-,-,-,-,-,-,-,...,0,0,0,0,0,0,0,0,에볼라바이러스병,001
3,5~9세,여,-,-,-,-,-,-,-,-,...,0,0,0,0,0,0,0,0,에볼라바이러스병,001
4,10~14세,남,-,-,-,-,-,-,-,-,...,0,0,0,0,0,0,0,0,에볼라바이러스병,001


### 저장

In [144]:
# fin_dat.to_csv('성별2001-2021.csv', index = False)

In [146]:
확인 = pd.read_csv('성별2001-2021.csv')
확인.shape # (479808, 25)
확인.head()

C:\Users\Admin\AppData\Local\Temp\ipykernel_3672\3355547775.py:1: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  확인 = pd.read_csv('성별2001-2021.csv')


(479808, 25)

,연령단위,성별,2001년,2002년,2003년,2004년,2005년,2006년,2007년,2008년,...,2014년,2015년,2016년,2017년,2018년,2019년,2020년,2021년,병,군구
0,0~4세,남,-,-,-,-,-,-,-,-,...,0,0,0,0,0,0,0,0,에볼라바이러스병,1
1,0~4세,여,-,-,-,-,-,-,-,-,...,0,0,0,0,0,0,0,0,에볼라바이러스병,1
2,5~9세,남,-,-,-,-,-,-,-,-,...,0,0,0,0,0,0,0,0,에볼라바이러스병,1
3,5~9세,여,-,-,-,-,-,-,-,-,...,0,0,0,0,0,0,0,0,에볼라바이러스병,1
4,10~14세,남,-,-,-,-,-,-,-,-,...,0,0,0,0,0,0,0,0,에볼라바이러스병,1
